# Notes:
##### This notebook demonstrates the preliminary method used to parse JSON from 4 web apis from a COVID-19 site. The data was then inserted en masse into our PostgreSQL database instance on AWS RDS. Since then, the process is being/has been migrated to Flask to include more streamlining through creation of classes for each table and a more organized architecture for our API services, unit tests, models, and images. 21 Apr 20

In [3]:
# Imports
import psycopg2
import requests
from psycopg2.extras import Json

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
##########################uslivestatus TABLE#######################################

In [0]:
# Connect to AWS RDS PostgreSQL DB instance via Psycopg - with secret password
conn = psycopg2.connect(host="database-earth.cftcdswr9dvc.us-east-2.rds.amazonaws.com", user="postgres", password=password)

In [0]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS uslivestatus;'

# create table query
create_table = '''
CREATE TABLE uslivestatus (
id SERIAL PRIMARY KEY,
countrycode TEXT,
province TEXT,
lat REAL,
lon REAL,
confirmed INTEGER,
deaths INTEGER,
recovered INTEGER,
active INTEGER,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

In [0]:
# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [0]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/live/country/us/status/confirmed")
record_list = response.json()

In [0]:
# How does it look?
record_list[:3]

[{'Active': 1551,
  'City': '',
  'CityCode': '',
  'Confirmed': 1621,
  'Country': 'United States of America',
  'CountryCode': 'US',
  'Date': '2020-04-13T00:00:00Z',
  'Deaths': 70,
  'Lat': '45.69',
  'Lon': '-93.9',
  'Province': 'Minnesota',
  'Recovered': 0},
 {'Active': 7990,
  'City': '',
  'CityCode': '',
  'Confirmed': 8225,
  'Country': 'United States of America',
  'CountryCode': 'US',
  'Date': '2020-04-13T00:00:00Z',
  'Deaths': 235,
  'Lat': '39.06',
  'Lon': '-76.8',
  'Province': 'Maryland',
  'Recovered': 0},
 {'Active': 797,
  'City': '',
  'CityCode': '',
  'Confirmed': 814,
  'Country': 'United States of America',
  'CountryCode': 'US',
  'Date': '2020-04-13T00:00:00Z',
  'Deaths': 17,
  'Lat': '41.13',
  'Lon': '-98.27',
  'Province': 'Nebraska',
  'Recovered': 0}]

In [0]:
# Separate keys and values. Drop city and citycode as they are empty
columns = [(list(x.keys()))[1:3] + (list(x.keys()))[5:] for x in record_list][0]
values = [(list(x.values()))[1:3] + (list(x.values()))[5:] for x in record_list]

In [0]:
# Check the columns
columns

['CountryCode',
 'Province',
 'Lat',
 'Lon',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active',
 'Date']

In [0]:
# Check some values
values[200]

['US',
 'New Hampshire',
 '43.45',
 '-71.56',
 1139,
 32,
 0,
 1107,
 '2020-04-16T00:00:00Z']

In [0]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

In [0]:
# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "uslivestatus"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

In [0]:
# Check how the execution of the insertion of many sql queries will look
sql_string[:250]

"INSERT INTO uslivestatus (CountryCode, Province, Lat, Lon, Confirmed, Deaths, Recovered, Active, Date)\nVALUES ('US', 'Minnesota', '45.69', '-93.9', 1621, 70, 0, 1551, '2020-04-13T00:00:00Z'),\n('US', 'Maryland', '39.06', '-76.8', 8225, 235, 0, 7990, '"

In [0]:
# Define our function to insert JSON data into database
def db_insertion():
    curs = conn.cursor()
    if curs != None:
        try:
            curs.execute( sql_string )
            conn.commit()
            print('\nfinished INSERT INTO execution')
        except Exception as error:
            print("\nexecute_sql() error:", error)
            conn.rollback()
        #close cursor and connection
        curs.close()

In [0]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [0]:
# Close our connection 
conn.close()

In [0]:
##########################uscases TABLE#######################################

In [0]:
# Connect to AWS RDS postgreSQL instance via psycopg
conn = psycopg2.connect(host="database-earth.cftcdswr9dvc.us-east-2.rds.amazonaws.com", user="postgres", password=password)

In [0]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS uscases;'

# create table query
create_table = '''
CREATE TABLE uscases (
id SERIAL PRIMARY KEY,
cases INTEGER,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [0]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/total/dayone/country/united-states/status/confirmed")
record_list = response.json()
# [a[5]]
# Separate keys and values (only the unique ones, no nulls, no uniform values)
columns = [(list(x.keys()))[7::2] for x in record_list][0]
values = [(list(x.values()))[7::2] for x in record_list]

In [0]:
# Check our columns
columns

['Cases', 'Date']

In [0]:
# Check some values
values[20]

[12, '2020-02-12T00:00:00Z']

In [0]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "uscases"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

len(sql_string)

2846

In [0]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [0]:
# Close our connection 
conn.close()

In [0]:
##########################summary TABLE#######################################

In [0]:
# Connect to AWS RDS postgreSQL instance via psycopg
conn = psycopg2.connect(host="database-earth.cftcdswr9dvc.us-east-2.rds.amazonaws.com", user="postgres", password=password)

In [0]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS summary;'

# create table query
create_table = '''
CREATE TABLE summary (
countrycode TEXT,
slug TEXT PRIMARY KEY,
newconfirmed INTEGER,
totalconfirmed INTEGER,
newdeaths INTEGER,
totaldeaths INTEGER,
newrecovered INTEGER,
totalrecovered INTEGER,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [0]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/summary")
list_of_dict = response.json()

# Just get country data
record_list = list_of_dict['Countries']

# Separate keys and values
columns = [(list(x.keys()))[1:] for x in record_list][0]
values = [(list(x.values()))[1:] for x in record_list]

In [0]:
columns

['CountryCode',
 'Slug',
 'NewConfirmed',
 'TotalConfirmed',
 'NewDeaths',
 'TotalDeaths',
 'NewRecovered',
 'TotalRecovered',
 'Date']

In [0]:
values[5]

['AD', 'andorra', 4, 717, 1, 37, 13, 248, '2020-04-21T03:15:26Z']

In [0]:
values[20]

['BY', 'belarus', 1485, 6264, 4, 51, 20, 514, '2020-04-21T03:15:26Z']

In [0]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "summary"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

len(sql_string)

17180

In [0]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [0]:
# Close our connection 
conn.close()

In [0]:
##########################uscounty TABLE#######################################

In [0]:
# Connect to AWS RDS postgreSQL instance via psycopg
conn = psycopg2.connect(host="database-earth.cftcdswr9dvc.us-east-2.rds.amazonaws.com", user="postgres", password=password)

In [0]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS uscounties;'

# create table query
create_table = '''
CREATE TABLE uscounties (
id SERIAL PRIMARY KEY,
country TEXT,
countrycode TEXT,
province TEXT,
city TEXT,
citycode TEXT,
lat REAL,
lon REAL,
cases INTEGER,
status TEXT,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [0]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/country/us/status/confirmed/live")
record_list = response.json()

# Separate keys and values
columns = [(list(x.keys())) for x in record_list][0]
values = [(list(x.values())) for x in record_list]

In [26]:
columns

['Country',
 'CountryCode',
 'Province',
 'City',
 'CityCode',
 'Lat',
 'Lon',
 'Cases',
 'Status',
 'Date']

In [27]:
values[20]

['United States of America',
 'US',
 'Wyoming',
 'Crook',
 '56011',
 '44.59',
 '-104.57',
 0,
 'confirmed',
 '2020-01-22T00:00:00Z']

In [28]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "uscounties"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

len(sql_string)

37571429

In [29]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [0]:
# Close our connection 
conn.close()